In [3]:
%load_ext autoreload
%autoreload 2
from glob import glob
import gc
import os
import sys
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_numeric_features, get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename
from func.time_utils import date_add_days
from func.ml_utils import save_feature, get_cnt_feature, get_dummie_feature, get_label_feature
from func.parallel_utils import get_parallel_arg_list
from joblib import delayed, Parallel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET, 'ProductCD']

train_paths = glob('../feature/eda_base/*_train.gz')
test_paths  = glob('../feature/eda_base/*_test.gz')

train_paths = [path for path in train_paths 
               if (path.count(COLUMN_DT) 
               or path.count(COLUMN_ID)
               or path.count(COLUMN_TARGET)
#                or path.count('_DT-')
#                or path.count('ProductCD_t')
               or path.count('D')
#                or path.count('C')
#                or path.count('card')
#                or path.count('addr')
#                or path.count('domain')
#                or path.count('uid')
               )
               and not path.count('fill')
               and not path.count('bin')
               and not path.count('uid')
               and not path.count('129')
              ]
test_paths = [path for path in test_paths 
               if (path.count(COLUMN_DT) 
               or path.count(COLUMN_ID)
               or path.count(COLUMN_TARGET)
#                or path.count('_DT-')
#                or path.count('ProductCD_t')
               or path.count('D')
#                or path.count('C')
#                or path.count('card')
#                or path.count('addr')
#                or path.count('domain')
#                or path.count('uid')
               )
               and not path.count('fill')
               and not path.count('bin')
               and not path.count('uid')
               and not path.count('129')
              ]

df_train = parallel_load_data(train_paths)
df_test = parallel_load_data(test_paths)
data = pd.concat([df_train, df_test], axis=0, ignore_index=True)
train_length = df_train.shape[0]
del df_train, df_test
gc.collect()

14

In [5]:
#========================================================================
# ProductCDあたりのC
#========================================================================
cols_V = [col for col in data.columns if col.startswith('D')]
cols_pcd = data['ProductCD'].unique()

for col in tqdm(cols_V):
    for pcd in cols_pcd:
        feature_name = f'{col}__ProductCD-{pcd}'
        data[feature_name] = -1
        data.loc[data['ProductCD'].isin([pcd]), feature_name] = data.loc[data['ProductCD'].isin([pcd]), col]

100%|██████████| 15/15 [00:18<00:00,  1.56s/it]


In [6]:
# sorted([col for col in data.columns if not col.count('C')])
cols_feature = sorted([col for col in data.columns if col.count('D') and col not in COLUMNS_IGNORE
                 and col.count('ProductCD-')
                ])

In [7]:
#========================================================================
# FE Aggregation User ID & TimeSeries Date
#========================================================================
# User別に期間を切って集計できる様にする
dir_save = 'check_trush'
        
train = data.iloc[:train_length]
test  = data.iloc[train_length:]

save_feature(train[cols_feature], '514', dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
save_feature(test[cols_feature],  '514', dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)

(590540,) | D10__ProductCD-C
(590540,) | D10__ProductCD-H
(590540,) | D10__ProductCD-R
(590540,) | D10__ProductCD-S
(590540,) | D10__ProductCD-W
(590540,) | D11__ProductCD-C
(590540,) | D11__ProductCD-H
(590540,) | D11__ProductCD-R
(590540,) | D11__ProductCD-S
(590540,) | D11__ProductCD-W
(590540,) | D12__ProductCD-C
(590540,) | D12__ProductCD-H
(590540,) | D12__ProductCD-R
(590540,) | D12__ProductCD-S
(590540,) | D12__ProductCD-W
(590540,) | D13__ProductCD-C
(590540,) | D13__ProductCD-H
(590540,) | D13__ProductCD-R
(590540,) | D13__ProductCD-S
(590540,) | D13__ProductCD-W
(590540,) | D14__ProductCD-C
(590540,) | D14__ProductCD-H
(590540,) | D14__ProductCD-R
(590540,) | D14__ProductCD-S
(590540,) | D14__ProductCD-W
(590540,) | D15__ProductCD-C
(590540,) | D15__ProductCD-H
(590540,) | D15__ProductCD-R
(590540,) | D15__ProductCD-S
(590540,) | D15__ProductCD-W
(590540,) | D1__ProductCD-C
(590540,) | D1__ProductCD-H
(590540,) | D1__ProductCD-R
(590540,) | D1__ProductCD-S
(590540,) | D1__Pr